In [8]:
import os

# A directory to be created for the results
folder = "auc_7_predictors"

try:
    os.mkdir(folder)
except OSError:
    print ("Creation of the directory %s failed (already exists?)" % folder)
else:
    print ("Successfully created the directory %s" % folder)

Successfully created the directory auc_7_predictors


In [9]:
# Load the rpy2 IPython extension into the notebook to use R with a magic command: %%R
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [10]:
%%R
# :::::::::
# R kernel
# :::::::::

install.packages("readxl")
install.packages("writexl")
install.packages("randomForestSRC")
install.packages("riskRegression")
install.packages("survival")
install.packages("cmprsk")
install.packages("prodlim")

_____________________________________________________________________________________________________________
Evaluate Cox cause-specific and Fine-Gray subdistribution hazard models, Random Forest on the validation data
_____________________________________________________________________________________________________________

In [11]:
%%R
# :::::::::
# R kernel
# :::::::::

library("readxl")
library("writexl")
library("randomForestSRC")
library("riskRegression")
library("survival")
library("cmprsk")
library("prodlim")

cohort <- read_excel("kihd_time_to_event.xlsx")
N_runs <- 3

for (i in seq(1,N_runs,1))
{
  print(i)
  
  kihd_train_file <- paste(c("splitting_7_predictors/training_valid_", i-1, ".xlsx"), collapse = "")
  kihd_test_file <- paste(c("splitting_7_predictors/validation_", i-1, ".xlsx"), collapse = "")
  
  kihd_train <- read_excel(kihd_train_file)
  kihd_test <- read_excel(kihd_test_file)
  
  colnames(kihd_train)[1] <- 'index'
  colnames(kihd_test)[1] <- 'index'
  
  kihd_train['time'] <- cohort[kihd_train$index + 1, 'time']/365.25
  kihd_test['time'] <- cohort[kihd_test$index + 1, 'time']/365.25
  
  kihd_train['event'] <- cohort[kihd_train$index + 1, 'event']
  kihd_test['event'] <- cohort[kihd_test$index + 1, 'event']
    
  ## --------------------------------------------------------------------------------------------------------------------------------
  ## Create interactions
  ## --------------------------------------------------------------------------------------------------------------------------------
  for (ii in seq(2,8))
  {
    if ((ii+1) <= 8)
      
      for (jj in seq(ii+1,8))
      {
        new_name = paste(c(colnames(kihd_train)[ii], colnames(kihd_train)[jj]), collapse = ".")
        kihd_train[new_name] = kihd_train[,colnames(kihd_train)[ii]] * kihd_train[,colnames(kihd_train)[jj]]
        kihd_test[new_name] = kihd_test[,colnames(kihd_test)[ii]] * kihd_test[,colnames(kihd_test)[jj]]
      }  
  }
  
  ## --------------------------------------------------------------------------------------------------------------------------------
  ## CVD output
  ## --------------------------------------------------------------------------------------------------------------------------------
  csc <- CSC(Hist(time,event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, data=kihd_train, cause=1)
  csc_interactions <- CSC(Hist(time,event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                            age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, data=kihd_train, cause=1)
  
  fgr <- FGR(Hist(time,event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, data=kihd_train, cause=1)
  fgr_interactions <- FGR(Hist(time,event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                             age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, data=kihd_train, cause=1)
  
  rf3 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
               kihd_train, splitrule = 'logrank', ntree = 250, 
               nodedepth = 3, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf3_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                              age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                            kihd_train, splitrule = 'logrank', ntree = 250, 
                            nodedepth = 3, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf5 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
               kihd_train, splitrule = 'logrank', ntree = 250, 
               nodedepth = 5, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf5_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                              age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                            kihd_train, splitrule = 'logrank', ntree = 250, 
                            nodedepth = 5, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf7 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
               kihd_train, splitrule = 'logrank', ntree = 250, 
               nodedepth = 7, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf7_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                              age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                            kihd_train, splitrule = 'logrank', ntree = 250, 
                            nodedepth = 7, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf9 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
               kihd_train, splitrule = 'logrank', ntree = 250, 
               nodedepth = 9, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf9_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                              age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                            kihd_train, splitrule = 'logrank', ntree = 250, 
                            nodedepth = 9, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf11 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
                kihd_train, splitrule = 'logrank', ntree = 250, 
                nodedepth = 11, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf11_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                               age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                             kihd_train, splitrule = 'logrank', ntree = 250, 
                             nodedepth = 11, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf13 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
                kihd_train, splitrule = 'logrank', ntree = 250, 
                nodedepth = 13, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf13_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                               age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                             kihd_train, splitrule = 'logrank', ntree = 250, 
                             nodedepth = 13, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf15 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
                kihd_train, splitrule = 'logrank', ntree = 250, 
                nodedepth = 15, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf15_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                               age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                             kihd_train, splitrule = 'logrank', ntree = 250, 
                             nodedepth = 15, na.action = "na.omit", case.wt = NULL, cause=1)
  
  score_test<-Score(list("Cause-specific Cox model"=csc,
                         "Fine-Gray model"=fgr,
                         "Cause-specific Cox model with interactions"=csc_interactions,
                         "Fine-Gray model with interactions"=fgr_interactions,
                         "Random Forest (depth = 3)" = rf3,
                         "Random Forest with interactions (depth = 3)" = rf3_interactions,
                         "Random Forest (depth = 5)" = rf5,
                         "Random Forest with interactions (depth = 5)" = rf5_interactions,
                         "Random Forest (depth = 7)" = rf7,
                         "Random Forest with interactions (depth = 7)" = rf7_interactions,
                         "Random Forest (depth = 9)" = rf9,
                         "Random Forest with interactions (depth = 9)" = rf9_interactions,
                         "Random Forest (depth = 11)" = rf11,
                         "Random Forest with interactions (depth = 11)" = rf11_interactions,
                         "Random Forest (depth = 13)" = rf13,
                         "Random Forest with interactions (depth = 13)" = rf13_interactions,
                         "Random Forest (depth = 15)" = rf15,
                         "Random Forest with interactions (depth = 15)" = rf15_interactions),
                    formula = Hist(time,event)~1,
                    data=kihd_test, times = seq(1,30,1), cause = 1, null.model = FALSE,  
                    plots = "calibration", metrics=c("auc","brier"),
                    summary = "risks")

  
  auc_result_file <- paste(c("auc_7_predictors/cvd_auc_validation_", i-1, ".xlsx"), collapse = "")
  write_xlsx(as.data.frame(score_test$AUC$score), auc_result_file)
  
  ## --------------------------------------------------------------------------------------------------------------------------------
  ## nonCVD output
  ## --------------------------------------------------------------------------------------------------------------------------------
  csc <- CSC(Hist(time,event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, data=kihd_train, cause=2)
  csc_interactions <- CSC(Hist(time,event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist +
                            age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, data=kihd_train, cause=2)
  
  fgr <- FGR(Hist(time,event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, data=kihd_train, cause=2)
  fgr_interactions <- FGR(Hist(time,event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist +
                             age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, data=kihd_train, cause=2)
  
  
  rf3 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
               kihd_train, splitrule = 'logrank', ntree = 250, 
               nodedepth = 3, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf3_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                              age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                            kihd_train, splitrule = 'logrank', ntree = 250, 
                            nodedepth = 3, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf5 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
               kihd_train, splitrule = 'logrank', ntree = 250, 
               nodedepth = 5, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf5_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                              age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                            kihd_train, splitrule = 'logrank', ntree = 250, 
                            nodedepth = 5, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf7 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
               kihd_train, splitrule = 'logrank', ntree = 250, 
               nodedepth = 7, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf7_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                              age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                            kihd_train, splitrule = 'logrank', ntree = 250, 
                            nodedepth = 7, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf9 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
               kihd_train, splitrule = 'logrank', ntree = 250, 
               nodedepth = 9, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf9_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                              age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                            kihd_train, splitrule = 'logrank', ntree = 250, 
                            nodedepth = 9, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf11 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
                kihd_train, splitrule = 'logrank', ntree = 250, 
                nodedepth = 11, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf11_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                               age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                             kihd_train, splitrule = 'logrank', ntree = 250, 
                             nodedepth = 11, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf13 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
                kihd_train, splitrule = 'logrank', ntree = 250, 
                nodedepth = 13, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf13_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                               age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                             kihd_train, splitrule = 'logrank', ntree = 250, 
                             nodedepth = 13, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf15 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
                kihd_train, splitrule = 'logrank', ntree = 250, 
                nodedepth = 15, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf15_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                               age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                             kihd_train, splitrule = 'logrank', ntree = 250, 
                             nodedepth = 15, na.action = "na.omit", case.wt = NULL, cause=2)
   
  score_test<-Score(list("Cause-specific Cox model"=csc,
                         "Fine-Gray model"=fgr,
                         "Cause-specific Cox model with interactions"=csc_interactions,
                         "Fine-Gray model with interactions"=fgr_interactions,
                         "Random Forest (depth = 3)" = rf3,
                         "Random Forest with interactions (depth = 3)" = rf3_interactions,
                         "Random Forest (depth = 5)" = rf5,
                         "Random Forest with interactions (depth = 5)" = rf5_interactions,
                         "Random Forest (depth = 7)" = rf7,
                         "Random Forest with interactions (depth = 7)" = rf7_interactions,
                         "Random Forest (depth = 9)" = rf9,
                         "Random Forest with interactions (depth = 9)" = rf9_interactions,
                         "Random Forest (depth = 11)" = rf11,
                         "Random Forest with interactions (depth = 11)" = rf11_interactions,
                         "Random Forest (depth = 13)" = rf13,
                         "Random Forest with interactions (depth = 13)" = rf13_interactions,
                         "Random Forest (depth = 15)" = rf15,
                         "Random Forest with interactions (depth = 15)" = rf15_interactions),
                    formula = Hist(time,event)~1,
                    data=kihd_test, times = seq(1,30,1), cause = 2, null.model = FALSE,  
                    plots = "calibration", metrics=c("auc","brier"),
                    summary = "risks")
  
  
  auc_result_file <- paste(c("auc_7_predictors/ncvd_auc_validation_", i-1, ".xlsx"), collapse = "")
  write_xlsx(as.data.frame(score_test$AUC$score), auc_result_file)
}


____________________________________________________________________________________________________________________
Evaluate Cox cause-specific and Fine-Gray subdistribution hazard models, Random Forest on the training and test data
____________________________________________________________________________________________________________________

In [12]:
%%R
# :::::::::
# R kernel
# :::::::::

library("readxl")
library("writexl")
library("randomForestSRC")
library("riskRegression")
library("survival")
library("cmprsk")
library("prodlim")

cohort <- read_excel("kihd_time_to_event.xlsx")

N_runs <- 3

for (i in seq(1,N_runs,1))
{
  print(i)
  
  kihd_train_file <- paste(c("splitting_7_predictors/training_", i-1, ".xlsx"), collapse = "")
  kihd_test_file <- paste(c("splitting_7_predictors/test_", i-1, ".xlsx"), collapse = "")
  
  kihd_train <- read_excel(kihd_train_file)
  kihd_test <- read_excel(kihd_test_file)
  
  colnames(kihd_train)[1] <- 'index'
  colnames(kihd_test)[1] <- 'index'
  
  kihd_train['time'] <- cohort[kihd_train$index + 1, 'time']/365.25
  kihd_test['time'] <- cohort[kihd_test$index + 1, 'time']/365.25
  
  kihd_train['event'] <- cohort[kihd_train$index + 1, 'event']
  kihd_test['event'] <- cohort[kihd_test$index + 1, 'event']
    
  
  ## --------------------------------------------------------------------------------------------------------------------------------
  ## Create interactions
  ## --------------------------------------------------------------------------------------------------------------------------------
  for (ii in seq(2,8))
  {
    if ((ii+1) <= 8)
      
      for (jj in seq(ii+1,8))
      {
        new_name = paste(c(colnames(kihd_train)[ii], colnames(kihd_train)[jj]), collapse = ".")
        kihd_train[new_name] = kihd_train[,colnames(kihd_train)[ii]] * kihd_train[,colnames(kihd_train)[jj]]
        kihd_test[new_name] = kihd_test[,colnames(kihd_test)[ii]] * kihd_test[,colnames(kihd_test)[jj]]
      }  
  }
  
  ## --------------------------------------------------------------------------------------------------------------------------------
  ## CVD output
  ## --------------------------------------------------------------------------------------------------------------------------------
  csc <- CSC(Hist(time,event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, data=kihd_train, cause=1)
  csc_interactions <- CSC(Hist(time,event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                            age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, data=kihd_train, cause=1)
  
  fgr <- FGR(Hist(time,event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, data=kihd_train, cause=1)
  fgr_interactions <- FGR(Hist(time,event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                             age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, data=kihd_train, cause=1)
  
  rf3 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
               kihd_train, splitrule = 'logrank', ntree = 250, 
               nodedepth = 3, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf3_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                              age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                            kihd_train, splitrule = 'logrank', ntree = 250, 
                            nodedepth = 3, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf5 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
               kihd_train, splitrule = 'logrank', ntree = 250, 
               nodedepth = 5, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf5_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                              age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                            kihd_train, splitrule = 'logrank', ntree = 250, 
                            nodedepth = 5, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf7 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
               kihd_train, splitrule = 'logrank', ntree = 250, 
               nodedepth = 7, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf7_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                              age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                            kihd_train, splitrule = 'logrank', ntree = 250, 
                            nodedepth = 7, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf9 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
               kihd_train, splitrule = 'logrank', ntree = 250, 
               nodedepth = 9, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf9_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                              age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                            kihd_train, splitrule = 'logrank', ntree = 250, 
                            nodedepth = 9, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf11 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
                kihd_train, splitrule = 'logrank', ntree = 250, 
                nodedepth = 11, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf11_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                               age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                             kihd_train, splitrule = 'logrank', ntree = 250, 
                             nodedepth = 11, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf13 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
                kihd_train, splitrule = 'logrank', ntree = 250, 
                nodedepth = 13, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf13_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                               age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                             kihd_train, splitrule = 'logrank', ntree = 250, 
                             nodedepth = 13, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf15 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
                kihd_train, splitrule = 'logrank', ntree = 250, 
                nodedepth = 15, na.action = "na.omit", case.wt = NULL, cause=1)
  
  rf15_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                               age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                             kihd_train, splitrule = 'logrank', ntree = 250, 
                             nodedepth = 15, na.action = "na.omit", case.wt = NULL, cause=1)
  
  score_train<-Score(list("Cause-specific Cox model"=csc,
                         "Fine-Gray model"=fgr,
                         "Cause-specific Cox model with interactions"=csc_interactions,
                         "Fine-Gray model with interactions"=fgr_interactions,
                         "Random Forest (depth = 3)" = rf3,
                         "Random Forest with interactions (depth = 3)" = rf3_interactions,
                         "Random Forest (depth = 5)" = rf5,
                         "Random Forest with interactions (depth = 5)" = rf5_interactions,
                         "Random Forest (depth = 7)" = rf7,
                         "Random Forest with interactions (depth = 7)" = rf7_interactions,
                         "Random Forest (depth = 9)" = rf9,
                         "Random Forest with interactions (depth = 9)" = rf9_interactions,
                         "Random Forest (depth = 11)" = rf11,
                         "Random Forest with interactions (depth = 11)" = rf11_interactions,
                         "Random Forest (depth = 13)" = rf13,
                         "Random Forest with interactions (depth = 13)" = rf13_interactions,
                         "Random Forest (depth = 15)" = rf15,
                         "Random Forest with interactions (depth = 15)" = rf15_interactions),
                    formula = Hist(time,event)~1,
                    data=kihd_train, times = seq(1,30,1), cause = 1, null.model = FALSE,  
                    plots = "calibration", metrics=c("auc","brier"),
                    summary = "risks")
  
  score_test<-Score(list("Cause-specific Cox model"=csc,
                         "Fine-Gray model"=fgr,
                         "Cause-specific Cox model with interactions"=csc_interactions,
                         "Fine-Gray model with interactions"=fgr_interactions,
                         "Random Forest (depth = 3)" = rf3,
                         "Random Forest with interactions (depth = 3)" = rf3_interactions,
                         "Random Forest (depth = 5)" = rf5,
                         "Random Forest with interactions (depth = 5)" = rf5_interactions,
                         "Random Forest (depth = 7)" = rf7,
                         "Random Forest with interactions (depth = 7)" = rf7_interactions,
                         "Random Forest (depth = 9)" = rf9,
                         "Random Forest with interactions (depth = 9)" = rf9_interactions,
                         "Random Forest (depth = 11)" = rf11,
                         "Random Forest with interactions (depth = 11)" = rf11_interactions,
                         "Random Forest (depth = 13)" = rf13,
                         "Random Forest with interactions (depth = 13)" = rf13_interactions,
                         "Random Forest (depth = 15)" = rf15,
                         "Random Forest with interactions (depth = 15)" = rf15_interactions),
                    formula = Hist(time,event)~1,
                    data=kihd_test, times = seq(1,30,1), cause = 1, null.model = FALSE,  
                    plots = "calibration", metrics=c("auc","brier"),
                    summary = "risks")
  
  
  auc_result_file <- paste(c("auc_7_predictors/cvd_auc_training_", i-1, ".xlsx"), collapse = "")
  write_xlsx(as.data.frame(score_train$AUC$score), auc_result_file)
  
  auc_result_file <- paste(c("auc_7_predictors/cvd_auc_test_", i-1, ".xlsx"), collapse = "")
  write_xlsx(as.data.frame(score_test$AUC$score), auc_result_file)
  
  ## --------------------------------------------------------------------------------------------------------------------------------
  ## nonCVD output
  ## --------------------------------------------------------------------------------------------------------------------------------
  csc <- CSC(Hist(time,event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, data=kihd_train, cause=2)
  csc_interactions <- CSC(Hist(time,event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist +
                            age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, data=kihd_train, cause=2)
  
  fgr <- FGR(Hist(time,event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, data=kihd_train, cause=2)
  fgr_interactions <- FGR(Hist(time,event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist +
                             age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, data=kihd_train, cause=2)
  
  
  rf3 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
               kihd_train, splitrule = 'logrank', ntree = 250, 
               nodedepth = 3, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf3_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                              age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                            kihd_train, splitrule = 'logrank', ntree = 250, 
                            nodedepth = 3, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf5 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
               kihd_train, splitrule = 'logrank', ntree = 250, 
               nodedepth = 5, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf5_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                              age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                            kihd_train, splitrule = 'logrank', ntree = 250, 
                            nodedepth = 5, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf7 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
               kihd_train, splitrule = 'logrank', ntree = 250, 
               nodedepth = 7, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf7_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                              age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                            kihd_train, splitrule = 'logrank', ntree = 250, 
                            nodedepth = 7, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf9 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
               kihd_train, splitrule = 'logrank', ntree = 250, 
               nodedepth = 9, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf9_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                              age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                            kihd_train, splitrule = 'logrank', ntree = 250, 
                            nodedepth = 9, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf11 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
                kihd_train, splitrule = 'logrank', ntree = 250, 
                nodedepth = 11, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf11_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                               age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                             kihd_train, splitrule = 'logrank', ntree = 250, 
                             nodedepth = 11, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf13 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
                kihd_train, splitrule = 'logrank', ntree = 250, 
                nodedepth = 13, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf13_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                               age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                             kihd_train, splitrule = 'logrank', ntree = 250, 
                             nodedepth = 13, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf15 <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist, 
                kihd_train, splitrule = 'logrank', ntree = 250, 
                nodedepth = 15, na.action = "na.omit", case.wt = NULL, cause=2)
  
  rf15_interactions <- rfsrc(Surv(time, event) ~ age + smoking + sbp + tc + hdl_c + diabetes + fam_hist + 
                               age.smoking + smoking.sbp + smoking.tc + smoking.hdl_c + smoking.fam_hist, 
                             kihd_train, splitrule = 'logrank', ntree = 250, 
                             nodedepth = 15, na.action = "na.omit", case.wt = NULL, cause=2)
  
  
  score_train<-Score(list("Cause-specific Cox model"=csc,
                         "Fine-Gray model"=fgr,
                         "Cause-specific Cox model with interactions"=csc_interactions,
                         "Fine-Gray model with interactions"=fgr_interactions,
                         "Random Forest (depth = 3)" = rf3,
                         "Random Forest with interactions (depth = 3)" = rf3_interactions,
                         "Random Forest (depth = 5)" = rf5,
                         "Random Forest with interactions (depth = 5)" = rf5_interactions,
                         "Random Forest (depth = 7)" = rf7,
                         "Random Forest with interactions (depth = 7)" = rf7_interactions,
                         "Random Forest (depth = 9)" = rf9,
                         "Random Forest with interactions (depth = 9)" = rf9_interactions,
                         "Random Forest (depth = 11)" = rf11,
                         "Random Forest with interactions (depth = 11)" = rf11_interactions,
                         "Random Forest (depth = 13)" = rf13,
                         "Random Forest with interactions (depth = 13)" = rf13_interactions,
                         "Random Forest (depth = 15)" = rf15,
                         "Random Forest with interactions (depth = 15)" = rf15_interactions),
                    formula = Hist(time,event)~1,
                    data=kihd_train, times = seq(1,30,1), cause = 2, null.model = FALSE,  
                    plots = "calibration", metrics=c("auc","brier"),
                    summary = "risks")
  
  
  score_test<-Score(list("Cause-specific Cox model"=csc,
                         "Fine-Gray model"=fgr,
                         "Cause-specific Cox model with interactions"=csc_interactions,
                         "Fine-Gray model with interactions"=fgr_interactions,
                         "Random Forest (depth = 3)" = rf3,
                         "Random Forest with interactions (depth = 3)" = rf3_interactions,
                         "Random Forest (depth = 5)" = rf5,
                         "Random Forest with interactions (depth = 5)" = rf5_interactions,
                         "Random Forest (depth = 7)" = rf7,
                         "Random Forest with interactions (depth = 7)" = rf7_interactions,
                         "Random Forest (depth = 9)" = rf9,
                         "Random Forest with interactions (depth = 9)" = rf9_interactions,
                         "Random Forest (depth = 11)" = rf11,
                         "Random Forest with interactions (depth = 11)" = rf11_interactions,
                         "Random Forest (depth = 13)" = rf13,
                         "Random Forest with interactions (depth = 13)" = rf13_interactions,
                         "Random Forest (depth = 15)" = rf15,
                         "Random Forest with interactions (depth = 15)" = rf15_interactions),
                    formula = Hist(time,event)~1,
                    data=kihd_test, times = seq(1,30,1), cause = 2, null.model = FALSE,  
                    plots = "calibration", metrics=c("auc","brier"),
                    summary = "risks")
  
  
    
  auc_result_file <- paste(c("auc_7_predictors/ncvd_auc_training_", i-1, ".xlsx"), collapse = "")
  write_xlsx(as.data.frame(score_train$AUC$score), auc_result_file)
  
  auc_result_file <- paste(c("auc_7_predictors/ncvd_auc_test_", i-1, ".xlsx"), collapse = "")
  write_xlsx(as.data.frame(score_test$AUC$score), auc_result_file)
}
